In [1]:
from mxnet import gluon, nd
from mxnet.gluon import nn

# 自定义了一个将输入减掉均值后输出的层
class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)

    def forward(self, x):
        return x - x.mean()
    

layer = CenteredLayer()
layer(nd.array([1, 2, 3, 4, 5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [2]:
net = nn.Sequential()
net.add(nn.Dense(128),
        CenteredLayer())

In [3]:
# 因为均值是浮点数，所以它的值是一个很接近0的数
net.initialize()
y = net(nd.random.uniform(shape=(4, 8)))
y.mean().asscalar()

-9.367795e-10

In [4]:
params = gluon.ParameterDict()
# 可以通过get函数从ParameterDict创建Parameter实例。
params.get('param2', shape=(2, 3))
params

(
  Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

In [5]:
class MyDense(nn.Block):
    # units为该层的输出个数，in_units为该层的输入个数
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

In [6]:
# 输入个数5，输出个数3
dense = MyDense(units=3, in_units=5)
dense.params

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

In [7]:
# 直接使用自定义层做前向计算。
dense.initialize()
# 随机两个参数，每个参数由5个组成
dense(nd.random.uniform(shape=(2, 5)))


[[0.06917784 0.01627153 0.01029644]
 [0.02602214 0.04537309 0.        ]]
<NDArray 2x3 @cpu(0)>

In [8]:
# 使用自定义层构造模型
net = nn.Sequential()
net.add(MyDense(8, in_units=64),
        MyDense(1, in_units=8))
net.initialize()
net(nd.random.uniform(shape=(2, 64)))


[[0.03820475]
 [0.04035058]]
<NDArray 2x1 @cpu(0)>

In [9]:
class MyDense(nn.Block):
    # units为该层的输出个数，in_units为该层的输入个数
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)
        
dense = MyDense(units=3, in_units=5)
dense.initialize()
dense(nd.random.uniform(shape=(1000, 5)))


[[9.3780458e-05 0.0000000e+00 7.4412957e-02]
 [1.1123408e-02 5.5128459e-02 1.3976939e-02]
 [1.8191710e-04 3.8248748e-03 1.5906963e-02]
 ...
 [0.0000000e+00 5.5083368e-02 3.4891628e-02]
 [0.0000000e+00 4.6397895e-03 7.5148061e-02]
 [2.7498063e-03 0.0000000e+00 8.7269217e-02]]
<NDArray 1000x3 @cpu(0)>